# Entendiendo el modelo GRAPE

El modelo está basado en GraphSage. La idea de este notebook es analizar la implementación de GRAPE.

<img src="notebook_figs/nb1.5_fig001.png" width="600">

En primer lugar vemos la implementacion del modelo. Se usa la libreria **Pytorch Geometric**.
En esta libreria se define la clase *MessagePassing* que permite implementar facilmente un esquema de paso de mensajes de la forma:

$$\mathbf{x}_i^{(k)} = \gamma^{(k)} \left( \mathbf{x}_i^{(k-1)}, \square_{j \in \mathcal{N}(i)} \, \phi^{(k)}\left(\mathbf{x}_i^{(k-1)}, \mathbf{x}_j^{(k-1)},\mathbf{e}_{j,i}\right) \right)$$

Lo único que tendremos que definir es:

- Un esquema de AGREGACIÓN ($\square_{j \in \mathcal{N}(i)}$) : Puede ser una función diferenciable, e invariante a la permitación, ej., sum, mean or max. Es el esquema de agregación que usamos. 
En nuestro caso, por defecto usamos la media (**mean**)
- $\phi$ : Es la función de paso de mensajes. Se define en la funcion **message**.
En nuestro caso:

$$\sigma (\mathbf{P} \cdot CONCAT(\mathbf{h}_u,\mathbf{e}_{uv})$$
- $\gamma$: Actualiza los embedings de cada nodo. Toma la salida de la AGREGACION  como primer argumento y cualquier argumento que se le pase a la función **propagate()**
En nuestro caso:

$$\sigma (\mathbf{Q} \cdot CONCAT(\mathbf{h}_u,\mathbf{n}_{v})$$

- Cuando se llama a **propagate()**, internamente se llama a la funciones **message()**, **aggregate()** y **update()**. Como argumentos básico se pasa **edge_index** y como adicionales pasamos todos los paremetros que necesiten las funciones anteriores.

## Modelo convolucional

In [330]:
import torch
from torch_geometric.nn.conv import MessagePassing


class EGraphSage(MessagePassing):
    """Non-minibatch version of GraphSage."""
    def __init__(self, 
                 in_channels,   #Dimension de entrada de los nodos
                 out_channels,  #Dimension de salida de los nodos
                 edge_channels, #Dimension de entrada de los arcos
                 activation,    #Funcion de activacion (Ej: RELU)
                 edge_mode,     #Forma en que se tratan los datos de arco. Esto hay que verlo
                 normalize_emb, #True o False. Si se normalizan los valores del embeding
                 aggr):         #Función de agregacion (ej: suma, media, max...)
        
        super(EGraphSage, self).__init__(aggr=aggr)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_channels = edge_channels
        self.edge_mode = edge_mode

     
        self.message_lin = nn.Linear(in_channels+edge_channels, 
                                     out_channels)
        self.agg_lin = nn.Linear(in_channels+out_channels, 
                                 out_channels)

        self.message_activation = get_activation(activation)
        self.update_activation = get_activation(activation)
        self.normalize_emb = normalize_emb
    
    def message(self, x_j, edge_attr):
        # x_j has shape [E, in_channels]
        # edge_index has shape [2, E]
        m_j = torch.cat((x_j, edge_attr),dim=-1)
        m_j = self.message_activation(self.message_lin(m_j))
        return m_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]
        # x has shape [N, in_channels]
        aggr_out = self.update_activation(self.agg_lin(torch.cat((aggr_out, x),dim=-1)))
        if self.normalize_emb:
            aggr_out = F.normalize(aggr_out, p=2, dim=-1) # Normaliza la salida (L2)
        return aggr_out
    
    # !Ojo! Llamamos a esta funcion cuando usamos la capa convolucional y le pasamos
    # los parámetros necesarios para el resto de funciones.
    def forward(self, x, 
                edge_attr, 
                edge_index):
        
        num_nodes = x.size(0)
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        return self.propagate(edge_index, 
                              x=x, 
                              edge_attr=edge_attr, 
                              size=(num_nodes, num_nodes))

    

El modelo completo (todas las capas) se genera usando la función **get_gnn**. Esta función devuelve una instancia de la clase **GNNStack**.

- model_types: Son los tipos de GNN en cada capa. Veamos varios ejemplos:
    * Ej de tres capas EGSAGE (el modelo analizado) --> Tendriamos una entrada de la forma **EGSAGE_EGSAGE_EGSAGE**

In [331]:
def get_gnn(data, args):
    model_types = args.model_types.split('_') # Lista con las capas y tipo.
    norm_embs = [True,]*len(model_types)      # Normaliza los embedings de todas las capas
    post_hiddens = [args.node_dim]
    
    # Construye las distintas capas del modelo (crea el modelo completo)
    model = GNNStack(data.num_node_features, # Atributos de nodo = Dimension del embeding del nodo = Atrib. de cada observacion
                     data.edge_attr_dim,     # Atributos de arco = Dimension del embeding del arco = 1
                     args.node_dim,          # La dimension del embeding de nodo a la salida de red(ej: 64)
                     args.edge_dim,          # La dimension del embeding de arco a la salida de red(ej: 64)
                     args.edge_mode,         # Determina como se opera con los arcos (default=1)
                     model_types,            # Tipo de layer. En nuestro caso EGSAGE (hay mas tipos disponibles)
                     args.dropout,           # Dropout de la MLP que actualiza el embedding de los nodos. 
                     args.gnn_activation,    # Funcion de activacion que usamos (ej: relu)
                     args.concat_states,     # T o F. Indica si se concatenan los embeddings de cada layer
                     post_hiddens,           # Capas ocultas de MLP que actualiza el embedding de los nodos. 
                     norm_embs,              # Lista bool. indicando si en una capa se normaliza embedding o no.
                     args.aggr)              # Funcion de agregación (mean, sum, max...)
    return model

In [332]:
from utils.utils import get_activation
import torch.nn as nn

class GNNStack(torch.nn.Module):
    def __init__(self, 
                 node_input_dim, 
                 edge_input_dim,
                 node_dim, 
                 edge_dim, 
                 edge_mode,
                 model_types, 
                 dropout, 
                 activation,
                 concat_states, 
                 node_post_mlp_hiddens,
                 normalize_embs, 
                 aggr):
        
        super(GNNStack, self).__init__()
        self.dropout = dropout
        self.activation = activation
        self.concat_states = concat_states
        self.model_types = model_types
        self.gnn_layer_num = len(model_types)

        # convs
        self.convs = self.build_convs(node_input_dim, 
                                      edge_input_dim,
                                      node_dim, 
                                      edge_dim, 
                                      edge_mode,
                                      model_types, 
                                      normalize_embs, 
                                      activation, 
                                      aggr)

        
        self.edge_update_mlps = self.build_edge_update_mlps(node_dim, 
                                                            edge_input_dim, 
                                                            edge_dim, 
                                                            self.gnn_layer_num, 
                                                            activation)
        
        # post node update
        self.node_post_mlp = self.build_node_post_mlp(node_dim, 
                                                      node_dim, 
                                                      node_post_mlp_hiddens, 
                                                      dropout, 
                                                      activation)

    def build_convs(self, node_input_dim, 
                    edge_input_dim,
                    node_dim, 
                    edge_dim, 
                    edge_mode,
                    model_types, 
                    normalize_embs, 
                    activation, aggr):
        
        convs = nn.ModuleList()
        
        # Primera capa convolucional
        conv = self.build_conv_model(model_types[0],
                                     node_input_dim,
                                     node_dim,
                                     edge_input_dim, 
                                     edge_mode, 
                                     normalize_embs[0], 
                                     activation, 
                                     aggr)
        convs.append(conv)
        
        # Resto de las capas convolucionales
        
        for l in range(1,len(model_types)):
            conv = self.build_conv_model(model_types[l],node_dim, node_dim,
                                    edge_dim, edge_mode, normalize_embs[l], activation, aggr)
            convs.append(conv)
        
        return convs
    
    def build_conv_model(self, model_type, 
                         node_in_dim, 
                         node_out_dim, 
                         edge_dim, 
                         edge_mode, 
                         normalize_emb, 
                         activation, 
                         aggr):
        return EGraphSage(node_in_dim, node_out_dim, edge_dim, activation, 
                          edge_mode, normalize_emb, aggr)
    
    def build_node_post_mlp(self, input_dim, 
                            output_dim, 
                            hidden_dims, 
                            dropout, 
                            activation):
        if 0 in hidden_dims:
            return get_activation('none')
        else:
            layers = []
            for hidden_dim in hidden_dims:
                layer = nn.Sequential(
                            nn.Linear(input_dim, hidden_dim),
                            get_activation(activation),
                            nn.Dropout(dropout),
                            )
                layers.append(layer)
                input_dim = hidden_dim
            layer = nn.Linear(input_dim, output_dim)
            layers.append(layer)
            return nn.Sequential(*layers)

    

    def build_edge_update_mlps(self, node_dim, edge_input_dim, edge_dim, gnn_layer_num, activation):
        edge_update_mlps = nn.ModuleList()
        edge_update_mlp = nn.Sequential(
                nn.Linear(node_dim+node_dim+edge_input_dim,edge_dim),
                get_activation(activation),
                )
        edge_update_mlps.append(edge_update_mlp)
        for l in range(1,gnn_layer_num):
            edge_update_mlp = nn.Sequential(
                nn.Linear(node_dim+node_dim+edge_dim,edge_dim),
                get_activation(activation),
                )
            edge_update_mlps.append(edge_update_mlp)
        return edge_update_mlps

    def update_edge_attr(self, x, edge_attr, edge_index, mlp):
        x_i = x[edge_index[0],:]
        x_j = x[edge_index[1],:]
        edge_attr = mlp(torch.cat((x_i,x_j,edge_attr),dim=-1))
        return edge_attr

    def forward(self, x, edge_attr, edge_index):
        
        for l, conv in enumerate(self.convs):
            
            # Actualiza estados (embedings) de los nodos
            x = conv(x, edge_attr, edge_index)
            # Actualiza embedings de los arcos
            edge_attr = self.update_edge_attr(x, edge_attr, edge_index, self.edge_update_mlps[l])
        
        x = self.node_post_mlp(x)
        return x

# Ejemplo ridículo

Contruimos una matriz de dos observaciones y 3 atributos

## Los datos

In [333]:
import numpy as np
import pandas as pd

from uci.uci_data import *

torch.cuda.manual_seed(0)

X=np.array([[1.0,2,3],[4,5,6]])
y=np.array([0,1])

df_X=pd.DataFrame(X,columns=["A1","A2","A3"],index=["O1","O2"])
df_y=pd.DataFrame(y)

df_X

,A1,A2,A3
O1,1.0,2.0,3.0
O2,4.0,5.0,6.0


In [334]:
#Definimos los parámetros del grafo y llamamos a la funcion que lo crea.
train_edge_prob= 0.7
train_y_prob= 0.7
seed=0
normalize=False

data = get_data(df_X, df_y, 0, train_edge_prob, 0, 
                    'y', train_y_prob, seed,normalize)

In [335]:
#Mostramos los arcos y sus valores

print(data.edge_index.numpy())
print(data.edge_attr.numpy().T)

[[0 0 0 1 1 1 2 3 4 2 3 4]
 [2 3 4 2 3 4 0 0 0 1 1 1]]
[[1. 2. 3. 4. 5. 6. 1. 2. 3. 4. 5. 6.]]


In [336]:
# Mostramos los valores iniciales de los embedings.
pd.DataFrame(data.x.numpy().T.astype(int), columns=["O1(n0)","O2(n1)","F1(n2)","F2(n3)","F3(n4)"])

,O1(n0),O2(n1),F1(n2),F2(n3),F3(n4)
0,1,1,1,0,0
1,1,1,0,1,0
2,1,1,0,0,1


Tenemos 5 nodos (n1,..., n5)

In [337]:
# Datos

#Valor inicial de los embedings 
x = data.x.clone().detach().to(device)

# Arcos de entrenamiento y su valor 
train_edge_index = data.train_edge_index.clone().detach().to(device)
train_edge_attr = data.train_edge_attr.clone().detach().to(device)
train_labels = data.train_labels.clone().detach().to(device)
test_input_edge_index = train_edge_index
test_input_edge_attr = train_edge_attr
test_input_edge_labels = train_labels

# Arcos de test y su valor.
test_edge_index = data.test_edge_index.clone().detach().to(device)
test_edge_attr = data.test_edge_attr.clone().detach().to(device)
test_labels = data.test_labels.clone().detach().to(device)

Los arcos del modelo con los que entrenamos son:

In [338]:
from torch_geometric.utils import to_dense_adj
print(to_dense_adj(train_edge_index,max_num_nodes=5).numpy())
print(train_edge_index)
print(train_labels)

[[[0. 0. 1. 0. 1.]
  [0. 0. 1. 1. 1.]
  [1. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [1. 1. 0. 0. 0.]]]
tensor([[0, 0, 1, 1, 1, 2, 4, 2, 3, 4],
        [2, 4, 2, 3, 4, 0, 0, 1, 1, 1]])
tensor([1., 3., 4., 5., 6.])


In [339]:
print(to_dense_adj(test_edge_index,max_num_nodes=5).numpy())
print(test_edge_index)
print(test_labels)

[[[0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]
tensor([[0, 3],
        [3, 0]])
tensor([2.])


## Definimos el modelo

In [340]:
#Pasamos los distintos parámetros del modelo
class paso_parametros:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

device='cpu'
parametros_modelo = {
    "model_types":'EGSAGE_EGSAGE',
    "node_dim": 6,
    "edge_dim": 6,
    "edge_mode":1,
    "gnn_activation":'relu',
    "concat_states":False,
    "dropout":0.,
    "aggr":'mean'
}
#Generamos el modelo y lo mostramos
model = get_gnn(data, paso_parametros(**parametros_modelo)).to(device)
# Imprimimos el modelo GNN
print(model.convs)

ModuleList(
  (0): EGraphSage(
    (message_lin): Linear(in_features=4, out_features=6, bias=True)
    (agg_lin): Linear(in_features=9, out_features=6, bias=True)
    (message_activation): ReLU()
    (update_activation): ReLU()
  )
  (1): EGraphSage(
    (message_lin): Linear(in_features=12, out_features=6, bias=True)
    (agg_lin): Linear(in_features=12, out_features=6, bias=True)
    (message_activation): ReLU()
    (update_activation): ReLU()
  )
)


In [341]:
def imprime_parametros(capa):
    sum_el=0
    for name, param in capa.named_parameters():
        if param.requires_grad:
            sum_el+=torch.numel(param)
            print ("Parametro:", name,"----> Tamaño:",param.shape)
    print("-----------")
    print("Parametros totales:", sum_el)

print("Parametros del modelo GNN")
print("-----------")
imprime_parametros(model.convs)
print("\nParametros del modelo Node_Post")
print("-----------")
imprime_parametros(model.node_post_mlp)


Parametros del modelo GNN
-----------
Parametro: 0.message_lin.weight ----> Tamaño: torch.Size([6, 4])
Parametro: 0.message_lin.bias ----> Tamaño: torch.Size([6])
Parametro: 0.agg_lin.weight ----> Tamaño: torch.Size([6, 9])
Parametro: 0.agg_lin.bias ----> Tamaño: torch.Size([6])
Parametro: 1.message_lin.weight ----> Tamaño: torch.Size([6, 12])
Parametro: 1.message_lin.bias ----> Tamaño: torch.Size([6])
Parametro: 1.agg_lin.weight ----> Tamaño: torch.Size([6, 12])
Parametro: 1.agg_lin.bias ----> Tamaño: torch.Size([6])
-----------
Parametros totales: 246

Parametros del modelo Node_Post
-----------
Parametro: 0.0.weight ----> Tamaño: torch.Size([6, 6])
Parametro: 0.0.bias ----> Tamaño: torch.Size([6])
Parametro: 1.weight ----> Tamaño: torch.Size([6, 6])
Parametro: 1.bias ----> Tamaño: torch.Size([6])
-----------
Parametros totales: 84


Definimos una capa de tarea. Nos da la prediccion a nivel de arco

In [342]:
from models.prediction_model import MLPNet
# Esta red neuronal MLP tradicional que predice valores de arcos
# La entrada son los embeddings de los extremos de un arco y la salida
# el valor del atributo.

impute_hiddens='6'
impute_hiddens = list(map(int,impute_hiddens.split('_')))
input_dim = parametros_modelo['node_dim'] * 2
output_dim = 1
impute_activation='relu'
impute_model = MLPNet(input_dim, output_dim,
                            hidden_layer_sizes=impute_hiddens,
                            hidden_activation=impute_activation,
                            dropout=parametros_modelo['dropout']).to(device)
impute_model

MLPNet(
  (layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=12, out_features=6, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.0, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=6, out_features=1, bias=True)
      (1): Identity()
    )
  )
)

# Entrenamiento

## Funciones de utilidad

Esta funcion genera un tensor con los arcos conocidos con una probabilidad **known_prob**. Se usa una distribucion uniforme.

### Función get_known_mask

In [343]:
def get_known_mask(known_prob, edge_num):
    known_mask = (torch.FloatTensor(edge_num, 1).uniform_() < known_prob).view(-1)
    return known_mask

In [184]:
#Ejemplo suponiendo una probabilidad del 20% y 20 elementos
ej1=get_known_mask(0.2, 20)
print(ej1)
print("\n El numero de arcos seleccionado es:",sum(ej1).item())

tensor([False, False,  True, False,  True, False, False,  True, False, False,
        False, False, False, False, False, False,  True,  True, False, False])

 El numero de arcos seleccionado es: 5


### Función mask_edge

Devuelve un grupo de arcos donde la máscara es True

In [344]:
def mask_edge(edge_index, edge_attr, mask, remove_edge):
    edge_index = edge_index.clone().detach()
    edge_attr = edge_attr.clone().detach()
    if remove_edge:
        edge_index = edge_index[:,mask]
        edge_attr = edge_attr[mask]
    else:
        edge_attr[~mask] = 0.
    return edge_index, edge_attr

In [200]:
# Hacemos pruebas de la función con el dataset de juguete

x = data.x.clone().detach().to(device)
train_edge_index = data.train_edge_index.clone().detach().to(device)
train_edge_attr = data.train_edge_attr.clone().detach().to(device)
train_labels = data.train_labels.clone().detach().to(device)

#Numero de arcos a entrenar
num_arcos=int(train_edge_attr.shape[0]/2)
print("Arcos a entrenar:",num_arcos)

# Sacamos los arcos conocidos con la funcion anterior
arcos_conocidos=get_known_mask(0.7, num_arcos)
print("Arcos conocidos:", arcos_conocidos.numpy())

known_edge_index, known_edge_attr = mask_edge(train_edge_index, train_edge_attr, 
                                              torch.cat((arcos_conocidos, arcos_conocidos), dim=0),True)
print("Arcos a entrenar Total:")
print(train_edge_index.numpy(), "\n",train_edge_attr.numpy().T)
print("Arcos despues de Drop:")
print(known_edge_index.numpy(), "\n",known_edge_attr.numpy().T)

Arcos a entrenar: 5
Arcos conocidos: [False  True  True  True  True]
Arcos a entrenar Total:
[[0 0 1 1 1 2 4 2 3 4]
 [2 4 2 3 4 0 0 1 1 1]] 
 [[1. 3. 4. 5. 6. 1. 3. 4. 5. 6.]]
Arcos despues de Drop:
[[0 1 1 1 4 2 3 4]
 [4 2 3 4 0 1 1 1]] 
 [[3. 4. 5. 6. 3. 4. 5. 6.]]


Con las dos funciones anteriores simulamos $\epsilon_{drop}=DropEdge(\epsilon, r_{drop})$ necesario para ejecutar el algoritmo. En nuestro caso $r_{drop}=1-known$

In [345]:
known=0.7 #Probabilidad de conocer el valor del atributo del arco (rdrop=1-known)

### Optimizador

In [346]:
import torch.optim as optim

#Creamos un optimizador Adam, lr=0.001 y weigh_decay=0.
filter_fn = filter(lambda p : p.requires_grad, list(model.parameters()))
opt=optimizer = optim.Adam(filter_fn, lr=0.001, weight_decay=0.)

### Entrenamiento

In [354]:
#Numero de epochs
epochs=2000

In [355]:
import torch.nn.functional as F

# Entrenamiento
for epoch in range(epochs):
    model.train()
    impute_model.train()
    
    # Obtenemos los arcos que usaremos para el entrenamiento
    known_mask = get_known_mask(known, int(train_edge_attr.shape[0] / 2)).to(device)
    double_known_mask = torch.cat((known_mask, known_mask), dim=0)
    known_edge_index, known_edge_attr = mask_edge(train_edge_index, train_edge_attr, 
                                                  double_known_mask, True)
    #################
    opt.zero_grad()
    
    # Calculamos el embeding del nodo
    x_embd = model(x, known_edge_attr, known_edge_index) # Dimensiones 519x64
    
    # Predecimos la etiqueta del arco
    pred = impute_model([x_embd[train_edge_index[0]], x_embd[train_edge_index[1]]]) #Dimensiones 9318x1
    pred_train = pred[:int(train_edge_attr.shape[0] / 2),0] # Dimensiones 4659
    
    # Calculamos la perdida del arco.
    label_train = train_labels
    loss = F.mse_loss(pred_train, label_train)
    loss.backward()
    opt.step()
    
print("Loss final:{}".format(loss))

Loss final:0.16727885603904724


# Vemos salidas del modelo

In [356]:
model.eval()
impute_model.eval()
with torch.no_grad():
    x_embd = model(x, test_input_edge_attr, test_input_edge_index)
    pred = impute_model([x_embd[test_edge_index[0], :], x_embd[test_edge_index[1], :]])
    pred_test = pred[:int(test_edge_attr.shape[0] / 2),0]
    label_test = test_labels

Imprimimos los embedings de salida

In [357]:
x_embd.T

tensor([[-1.3663, -4.9983, -1.2062, -4.9983, -4.9983],
        [ 3.2973,  9.3100,  2.9202,  9.3100,  9.3100],
        [-2.9129, -8.9212, -2.5918, -8.9212, -8.9212],
        [-3.3403, -9.4328, -2.7771, -9.4328, -9.4328],
        [ 2.9541, 10.2637,  2.4039, 10.2637, 10.2637],
        [-3.0347, -6.9766, -2.1346, -6.9766, -6.9766]])

In [367]:
print(test_edge_index[0])
print(test_edge_index[1])
print(x_embd[test_edge_index[0], :].T)

tensor([0, 3])
tensor([3, 0])
tensor([[-1.3663, -4.9983],
        [ 3.2973,  9.3100],
        [-2.9129, -8.9212],
        [-3.3403, -9.4328],
        [ 2.9541, 10.2637],
        [-3.0347, -6.9766]])


In [369]:
pred

tensor([[2.6118],
        [4.4722]])

Es curioso que se quedan con el primer valor....

In [379]:
pred[:int(test_edge_attr.shape[0] / 2),0]

tensor([2.6118])

In [380]:
# La etiqueta real del arco seria
label_test

tensor([2.])